<a href="https://colab.research.google.com/github/anbebe/AlgonautsChallenge/blob/main/Label_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)
data_dir = '/content/drive/MyDrive/algonauts_2023_tutorial_data' 
parent_submission_dir = '/content/drive/MyDrive/algonauts_2023_challenge_submission' 

subj = 1

class argObj:
  def __init__(self, data_dir, parent_submission_dir, subj):
    
    self.subj = format(subj, '02')
    self.data_dir = os.path.join(data_dir, 'subj'+self.subj)
    self.parent_submission_dir = parent_submission_dir
    self.subject_submission_dir = os.path.join(self.parent_submission_dir,
        'subj'+self.subj)

    # Create the submission directory if not existing
    if not os.path.isdir(self.subject_submission_dir):
        os.makedirs(self.subject_submission_dir)

args = argObj(data_dir, parent_submission_dir, subj)

train_img_dir  = os.path.join(args.data_dir, 'training_split', 'training_images')
test_img_dir  = os.path.join(args.data_dir, 'test_split', 'test_images')

img_height = 320
img_width = 320
batch_size = 100

Mounted at /content/drive/


In [18]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_img_dir,
  color_mode="rgb",
  labels=None,
  shuffle=False,
  seed=None,
  image_size=(img_height, img_width),
  batch_size=None)

val_ds = tf.keras.utils.image_dataset_from_directory(
  test_img_dir,
  color_mode="rgb",
  labels=None,
  shuffle=False,
  seed=None,
  image_size=(img_height, img_width),
  batch_size=None)

Found 9841 files belonging to 1 classes.
Found 159 files belonging to 1 classes.


In [3]:
fmri_dir = os.path.join(args.data_dir, 'training_split', 'training_fmri')
lh_fmri = np.load(os.path.join(fmri_dir, 'lh_training_fmri.npy'))
rh_fmri = np.load(os.path.join(fmri_dir, 'rh_training_fmri.npy'))

In [19]:
for element in train_ds:
    print("image shape: ", element.shape)
    break

print("\n", "After enumeration:","\n")

#Enumerate Dataset. This will be used to map the labels to the images
en_train = train_ds.enumerate(start=1)

for element in en_train:
    print("image shape: ", element[1].shape)
    print("the index: ", element[0])
    print("tuple length: ", len(element))
    break

lh_fmri_tensor = tf.constant(lh_fmri)
rh_fmri_tensor = tf.constant(rh_fmri)

def add_fmri(indx,img):

    index = int(indx)
    img = img #320x320x3
    
    lh_fmri = tf.gather(lh_fmri_tensor, index)
    rh_fmri = tf.gather(rh_fmri_tensor, index)

    return img, lh_fmri, rh_fmri

train_ds = en_train.map(add_fmri)

image shape:  (320, 320, 3)

 After enumeration: 

image shape:  (320, 320, 3)
the index:  tf.Tensor(1, shape=(), dtype=int64)
tuple length:  2


In [20]:
for element in train_ds:
    print("image shape: ", element[0].shape)
    print("lh_fmri: ", element[1])
    print("rh_fmri: ", element[2])
    print("tuple length: ", len(element))
    break

image shape:  (320, 320, 3)
lh_fmri:  tf.Tensor(
[ 0.72242236 -0.2831422   0.92416    ...  1.1387796   1.1579975
  1.3964182 ], shape=(19004,), dtype=float32)
rh_fmri:  tf.Tensor(
[ 0.5131692   0.98874855 -0.05418793 ...  1.4851211   1.6171584
  1.6620398 ], shape=(20544,), dtype=float32)
3
